In [ ]:
import star_functions as nana

In [ ]:
import importlib
importlib.reload(nana)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import ascii
import numpy as np
from astropy import units as u

In [ ]:
star_files = glob.glob("*.dat")

In [ ]:
print(star_files)
#kplr010536147

In [ ]:
def do_one_star(star_fn, exptime = 'long'):
    
    star = star_fn[:13] #brittle

    print("working on ", star)
    
    #pipeline 
    lc, delta_f, sampling_time, exptime = nana.star(star, exptime = exptime)
    print("delta_f:", delta_f)
    print("sampling_time:", sampling_time)
    if lc is None:
        return 
    f_max = (3 / (2*sampling_time))
    f_min = delta_f/3
    t_fit, flux_fit, weight_fit = nana.mask_vals(lc)
    freq_full, power_full = nana.pg_full(f_min, f_max, lc)
    freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
    indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
    refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
    fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
    regions, final_freqs, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit, flux_fit, weight_fit, exptime)
    '''
    plt.plot(freq_full, power_full, 'k.')
    plt.scatter(final_freqs, refined_power, color = 'red', marker = 'o')
    plt.show()
    
    plt.plot(freq_mini, power_mini, 'g-')
    plt.semilogy()
    plt.show()
    '''
    a_early, a_late, b_early, b_late = nana.check_coherence(t_fit, flux_fit, weight_fit, final_freqs, exptime)
    rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit)
    delta_chi2s = nana.null_chi_squared(flux_fit, weight_fit) - chi2s
    sharpnesses = nana.sharpness(second_derivatives, refined_power)
    
    '''
    data = Table()


    data['modes'] = final_freqs
    data['regions'] = regions
    data['corresponding_freq_in_region_A'] = refined_freq
    data['sharpness'] = sharpnesses
    data['delta_chi_squares'] = delta_chi2s
    data['rate_of_change_of_phase'] = rate_of_phase
    data['rate_of_change_of_amplitude'] = rate_of_amp
    
    ascii.write(
        data,
        star + '.csv',
        overwrite=True,
        format="csv",
        formats={
            "modes": "{:.10f}",
            "corresponding_freq_in_region_A": "{:.5f}",
            "sharpness": "{:.7e}",
            "delta_chi_squares": "{:.7e}",
            "rate_of_change_of_phase": "{:.7e}",
            "rate_of_change_of_amplitude": "{:.7e}"
        }
    )
    '''

    return
    

In [ ]:
##out of spec stars
#kplr006462033
#kplr010536147
#kplr008459899
#kplr003459297
#kplr003240411
#kplr012258330
#

In [ ]:
#out of spec stars
#kplr006462033
#kplr010536147
#kplr008459899
#kplr003240411
#kplr012258330
#kplr008766405

In [ ]:
#for star in star_files:
  #  do_one_star(star, exptime = exptime)

In [ ]:
def check_coherence2(ts, ys, weights, oms, T):

    Qs = [[[ ] , [ ]] for i in range(4)]

    ts_median = np.median(ts)
    t_first = np.median(ts[ts < ts_median])
    t_third = np.median(ts[ts >= ts_median])

    mask_q1 = (ts < ts_median) & (ts < t_first)
    mask_q2 = (ts < ts_median) & (ts >= t_first)
    mask_q3 = (ts >= ts_median) & (ts < t_third)
    mask_q4 = (ts >= ts_median) & (ts >= t_third)

    cases = [mask_q1, mask_q2, mask_q3, mask_q4]
    
    for om in oms:
        for q, I in zip(Qs, cases):
            A = nana.integral_design_matrix(ts[I], om, T)
            pars, _ =nana.weighted_least_squares_new(A, ys[I], weights[I])
            q[0].append(pars[1])
            q[1].append(pars[2])

    return Qs[0], Qs[1], Qs[2], Qs[3]
            



In [ ]:
#og KIC 520 star star
lc, delta_f, sampling_time, exptime520 = nana.star('KIC 003336252')
print("delta_f:", delta_f)
print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit520, flux_fit520, weight_fit520 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs520, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

plt.plot(freq_full, power_full, 'k.')
plt.scatter(final_freqs520, refined_power, color = 'red', marker = 'o')
plt.show()

plt.plot(freq_mini, power_mini, 'g-')
plt.semilogy()
plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit520)
delta_chi2s = nana.null_chi_squared(flux_fit520, weight_fit520) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
#og KIC 520 star star
lc, delta_f, sampling_time, exptime520 = nana.star('KIC 5202905')
print("delta_f:", delta_f)
print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit520, flux_fit520, weight_fit520 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs520, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

plt.plot(freq_full, power_full, 'k.')
plt.scatter(final_freqs520, refined_power, color = 'red', marker = 'o')
plt.show()

plt.plot(freq_mini, power_mini, 'g-')
plt.semilogy()
plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit520)
delta_chi2s = nana.null_chi_squared(flux_fit520, weight_fit520) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
def splitting(ts, K, jackknife = True):
    '''
    # splitting()
    Produce either disjoint or jackknife subsamples.
    
    ## Inputs:
    - `ts`: shape `(N, )` array of times (not necesarily ordered)
    - `K`: number of subsamples to return
    - `jackknife`: `True` for jackknife subsamples; otherwise disjoint
    
    ## Output: 
    - `masks`: shape `(K, N)` boolean array; `True` if in subsample

    ## Comment:
    - Relies on numpy conventions about relationships between integers and bools.
    '''
    N = len(ts)
    indices = np.argsort(ts)
    split_indices = np.array_split(indices, K)
    if jackknife:
        masks = np.ones((K, N), dtype=bool) # make everything True by default
        for i, idx in enumerate(split_indices):
            masks[i, idx] = False # remove subsample
    else:
        masks = np.zeros((K, N), dtype=bool) # make everything False by default
        for i, idx in enumerate(split_indices):
            masks[i, idx] = True # add subsample
    return masks

In [ ]:
all, half, quartiles, eighths = coherence_all(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(16, 9))

for idx, (ax, points1, points2, points3, points4) in enumerate(zip(axes.flat, all, half, quartiles, eighths)):
    a_all = points1[0]  
    b_all = points1[1]  

    a_half = points2[:, 0]  
    b_half = points2[:, 1]  

    a_quarter = points3[:, 0]  
    b_quarter = points3[:, 1] 

    a_eighth = points4[:, 0]
    b_eighth= points4[:, 1]

    ax.scatter(a_half, b_half, color='green', marker='*', label='Halves')
    ax.scatter(a_quarter, b_quarter, color='blue', marker='+', label='Quarters')
    ax.scatter(a_eighth, b_eighth, color='k', marker='.', label='Eighths (Jackknife)')
    ax.scatter(a_all, b_all, color='red', marker='x', label='All Data')

    if idx == 0:
        ax.legend()
    
    ax.axvline(0, color = 'k')
    ax.axhline(0, color = 'k')
    ax.set_title(final_freqs520[idx])
    ax.set_xlabel("a points")
    ax.set_ylabel("b points")
    ax.grid(True)
    ax.ticklabel_format(style='sci', scilimits=(-3, 3), axis='both')
    ax.axis('equal')



plt.tight_layout()


In [ ]:
###FINAL COHERENCE TEST!!!!
def coherence_all(ts, ys, weights, final_freq, T):
    oms = np.array([f * 2 * np.pi for f in final_freq])
    all = np.zeros((len(oms), 2))

    for idx, om in enumerate(oms):
        A = nana.integral_design_matrix(ts, om, T)
        pars, _ = nana.weighted_least_squares_new(A, ys, weights)
        all[idx][0] = pars[1]
        all[idx][1] = pars[2]
        

    splits = np.array([2, 4, 8])
    results = [np.zeros((len(oms), n, 2)) for n in splits]
    
    for split, result in zip(splits, results):     

        jack = (split == 8)
        masks = splitting(ts, split, jackknife = jack)
        for idx, om in enumerate(oms):
            for i, mask in enumerate(masks):
                A = nana.integral_design_matrix(ts[mask], om, T)
                pars, _ = nana.weighted_least_squares_new(A, ys[mask], weights[mask])
                result[idx][i][0] = pars[1] #a
                result[idx][i][1] = pars[2] #b

    return all, results[0], results[1], results[2] #all, half, quarter, eighth (+jacknives)
    

In [ ]:
## rewriting code for rate of change of phase, rate of change of log A

half

a_halves = half[0][:, 0]
b_halves = half[0][:, 1]


delta_r = [a_latte - a_earl_grey, b_latte - b_earl_grey]
        vector_r = [0.5 * (a_latte + a_earl_grey), 0.5 * (b_latte + b_earl_grey)]

        cross_z = delta_r[0] * vector_r[1] - delta_r[1] * vector_r[0]
        dot_r = np.dot(vector_r, vector_r)

        if dot_r == 0:
            raise ZeroDivisionError(f"change_in_phase_and_amp(): phase vector has zero magnitude at index {i}")

        rates_of_phases[i] = (1 / delta_t) * (cross_z / dot_r)
        rates_of_amps[i]   = (1 / delta_t) * (np.dot(delta_r, vector_r) / dot_r)

In [ ]:

for ab_pair in half:
    
a_halves = half[0][:, 0]
b_halves = half[0][:, 1]
print(a_halves, b_halves)

In [ ]:
##We did the two split now this is a function for the four split

#Get a,b values for a four split
#compute the deltak which is the separation with the "full" a,b values with the individual 4
#Compute x hat and y hat
#Then we are given two formulas to approximate values
#compute the 1/2 rms

a_all = all[0][0] # a for first freq
b_all = all[0][1] # b for first freq

a_quarters = quartiles[0][:, 0]
b_quarters = quartiles[0][:, 1]

deltak = np.zeros((4,2))

for i, (a,b) in enumerate(zip(a_quarters, b_quarters)):   
    deltak[i] = [a - a_all, b - b_all]

x_not = [a_all, b_all]
x_not_norm = np.linalg.norm(x_not)
x_hat = x_not / x_not_norm
y_hat = [x_hat[1], - x_hat[0]]

amp_change = np.dot(deltak, x_hat)/ x_not_norm
phase_change = np.dot(deltak, y_hat)/x_not_norm

print(f"this is amp change{amp_change} and this is phase change{phase_change}")
sigma_lnA = 0.5 * np.sqrt((amp_change ** 2))#1/2 rms lnA
sigma_phi = 0.5 * np.sqrt((phase_change ** 2)) #1/2 rms phase

print(sigma_lnA, sigma_phi)

In [ ]:
###jackknife sampling to dimensionless


a_jacks = eighths[0][:, 0]
b_jacks = eighths[0][:, 1]

deltak = np.zeros((8,2))

for i, (a,b) in enumerate(zip(a_jacks, b_jacks)):   
    deltak[i] = [a - a_all, b - b_all]

phase_change = np.dot(deltak, y_hat)/x_not_norm

rms_jack = np.sqrt((7/8) * np.sum(phase_change ** 2)) #sigma phi 
    

In [ ]:
## inputs are the quarters, half, jacks, all results

#compute two values for the a,b half

#compute the amp change and the phase change values (4)

#compute the varicnes for lnA, and phi (4)

#comput eth ephase change and its varience(jack)

#return 2 + 2 + 2+ 1

In [ ]:
#

def sampling_stats(alls, quartiles, eighths):

    f_num = len(alls)
    
    amp_change = np.zeros((f_num, 4))
    phase_change = np.zeros((f_num, 4))

    sigma_lnA = np.zeros(f_num)
    sigma_phi4 = np.zeros(f_num)
    sigma_phij = np.zeros(f_num)
    
    for inx, (all, quartile, eighth) in enumerate(zip(alls, quartiles, eighths)):

        deltak_4 = np.zeros((4,2))
        deltak_j = np.zeros((8,2))
        a,b = all[0], all[1]
    
        for i, q in enumerate(quartile):   
            deltak_4[i] = [a - q[0], b - q[1]]

        for i, j in enumerate(eighth):   
            deltak_j[i] = [a - j[0], b - j[1]]
                
        x = np.array([a,b])
        x_norm = np.linalg.norm([a, b])
        x_hat = x / x_norm
        y_hat = [x_hat[1], - x_hat[0]]

        amp = np.dot(deltak_4, x_hat)/ x_norm
        phase4 = np.dot(deltak_4, y_hat)/x_norm
        phasej = np.dot(deltak_j, y_hat)/x_norm

        var_lnA = 0.5 * np.sqrt(np.mean(amp ** 2))#1/2 rms lnA
        varphi4 = 0.5 * np.sqrt(np.mean(phase4 ** 2)) #1/2 rms phase

        varphij = np.sqrt((7/8) * np.sum(phasej ** 2)) #sigma phi

        amp_change[inx] = amp
        phase_change[inx] = phase4

        sigma_lnA[inx] = var_lnA
        sigma_phi4[inx] = varphi4
        sigma_phij[inx] = varphij 
        
        

    return amp_change, phase_change, sigma_lnA, sigma_phi4, sigma_phij

In [ ]:
amp_change, phase_change, sigma_lnA, sigma_phi4, sigma_phij = sampling_stats(all, quartiles, eighths)
print(f"this is the change in ln(amp){amp_change}")
print(f"this is the change in phase{phase_change}")
print("sigma lnA:", sigma_lnA)
print("sigma phi for quarter:", sigma_phi4)
print("sigma phi for jackknife:", sigma_phij)

In [ ]:
a_all = all[i][0] # a for first freq
        b_all = all[i][1]

        a_quarters = quartiles[0][:, 0]
        b_quarters = quartiles[0][:, 1]

        a_jacks = eighths[0][:, 0]
        b_jacks = eighths[0][:, 1]

        deltak_4 = np.zeros((4,2))
        deltak_j = np.zeros((8,2))
    
        for i, (a,b) in enumerate(zip(a_quarters, b_quarters)):   
            deltak_4[i] = [a - a_all, b - b_all]

        for i, (a,b) in enumerate(zip(a_jacks, a_jacks)):   
            deltak_j[i] = [a - a_all, b - b_all]

        x_not = [a_all, b_all]
        x_not_norm = np.linalg.norm(x_not)
        x_hat = x_not / x_not_norm
        y_hat = [x_hat[1], - x_hat[0]]

        amp_change = np.dot(deltak_4, x_hat)/ x_not_norm
        phase_change_4 = np.dot(deltak_4, y_hat)/x_not_norm
        phase_change_j = np.dot(deltak_j, y_hat)/x_not_norm

        sigma_lnA = 0.5 * np.sqrt((phase_change_4 ** 2))#1/2 rms lnA
        sigma_phi_4 = 0.5 * np.sqrt((phase_change_4 ** 2)) #1/2 rms phase

        sigma_phi_j = np.sqrt((7/8) * np.sum(phase_change ** 2)) #sigma phi 


#another function?:

In [ ]:
from astropy.table import Table
from astropy.io import ascii